In [1]:
import json
import pymupdf
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import os


ModuleNotFoundError: No module named 'langchain'

In [ ]:

# Set up embeddings using Hugging Face
def get_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")



In [2]:

# Initialize Vector Store
def init_vector_store(persist_directory="./db"):
    embeddings = get_embeddings()
    return Chroma(collection_name="test_store", embedding_function=embeddings, persist_directory=persist_directory)



# Load PDF and extract text
def process_pdf(pdf_path):
    docs = []
    pdf_document = fitz.open(pdf_path)  # Open PDF

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        text = page.get_text("text")  # Extract text

        if text.strip():  # Only store non-empty pages
            docs.append({"content": text, "metadata": {"page": page_num + 1}})

    return docs



In [3]:

# Load JSON file
def process_json(json_path):
    docs = []
    with open(json_path, "r") as file:
        data = json.load(file)
        for idx, item in enumerate(data):
            docs.append({"content": json.dumps(item), "metadata": {"id": idx + 1}})
    return docs
    


In [4]:

# Function to split text into chunks
def split_documents(docs, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    formatted_docs = [doc["content"] for doc in docs]
    return text_splitter.create_documents(formatted_docs, metadatas=[doc["metadata"] for doc in docs])


In [5]:

# Ingest data into ChromaDB
def ingest_data(pdf_path="test.pdf", json_path="test.json", persist_directory="./db"):
    vector_db = init_vector_store(persist_directory)
    
    # Process documents
    pdf_docs = process_pdf(pdf_path)
    json_docs = process_json(json_path)
    all_docs = pdf_docs + json_docs

    # Chunking the documents
    split_docs = split_documents(all_docs)

    # Add documents to vector DB
    vector_db.add_documents(split_docs)
    vector_db.persist()  # Save to disk

    return vector_db
    

In [6]:
import fitz

pdf_path = "data/AI_and_Technology_Knowledge_Base.pdf"  # Replace with your file

def test_pdf_processing():
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc[page_num]
        print(f"\n--- Page {page_num + 1} ---\n")
        print(page.get_text("text"))  # Extract and print text

test_pdf_processing()


--- Page 1 ---

AI and Technology Knowledge Base
Introduction to Artificial Intelligence
Artificial Intelligence (AI) is a branch of computer science that focuses on creating machines
capable of performing tasks that would normally require human intelligence. These tasks
include learning, problem-solving, perception, language understanding, and
decision-making. AI systems can be classified into two broad categories: narrow AI, which is
designed to perform a narrow task (such as facial recognition or internet searches), and
general AI, which has the ability to perform any intellectual task that a human can do. AI is
transforming industries, from healthcare and finance to transportation and manufacturing.
The evolution of AI can be traced back to early philosophical discussions about the nature of
the mind and mechanical reasoning. However, the modern development of AI began with the
invention of computers, which made it possible to simulate reasoning and decision-making
through algorit